In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_data']
    train_labels = save['train_labels']
    test_dataset = save['test_data']
    test_labels = save['test_labels']
    test_filenames = save['test_filenames']
    valid_dataset = save['valid_data']
    valid_labels = save['valid_labels']
    del save
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (73401, 32, 32, 1) (73401, 6)
Validation set (10000, 32, 32, 1) (10000, 6)
Test set (13068, 32, 32, 1) (13068, 6)


In [3]:
def gaussian_filter(kernel_shape):
    x = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)
    
    for kernel_idx in xrange(0, kernel_shape[2]):
        for i in xrange(0, kernel_shape[0]):
            for j in xrange(0, kernel_shape[1]):
                x[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
    
    return tf.convert_to_tensor(x / np.sum(x), dtype=tf.float32)

In [4]:
def gauss(x, y, sigma=3.0):
    Z = 2 * np.pi * sigma ** 2
    return  1. / Z * np.exp(-(x ** 2 + y ** 2) / (2. * sigma ** 2))

In [5]:
def LecunLCN(X, image_shape, threshold=1e-4, radius=7, use_divisor=True):
    """Local Contrast Normalization"""
    """[http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf]"""

    # Get Gaussian filter
    filter_shape = (radius, radius, image_shape[3], 1)

    #self.filters = theano.shared(self.gaussian_filter(filter_shape), borrow=True)
    filters = gaussian_filter(filter_shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    # Compute the Guassian weighted average by means of convolution
    convout = tf.nn.conv2d(X, filters, [1,1,1,1], 'SAME')

    # Subtractive step
    mid = int(np.floor(filter_shape[1] / 2.))

    # Make filter dimension broadcastable and subtract
    centered_X = tf.sub(X, convout)

    # Boolean marks whether or not to perform divisive step
    if use_divisor:
        # Note that the local variances can be computed by using the centered_X
        # tensor. If we convolve this with the mean filter, that should give us
        # the variance at each point. We simply take the square root to get our
        # denominator

        # Compute variances
        sum_sqr_XX = tf.nn.conv2d(tf.square(centered_X), filters, [1,1,1,1], 'SAME')

        # Take square root to get local standard deviation
        denom = tf.sqrt(sum_sqr_XX)

        per_img_mean = tf.reduce_mean(denom)
        divisor = tf.maximum(per_img_mean, denom)
        # Divisise step
        new_X = tf.truediv(centered_X, tf.maximum(divisor, threshold))
    else:
        new_X = centered_X
    return new_X

In [6]:
def accuracy(predictions, labels, printstat=False):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [11]:
image_size = 32
num_labels = 11 # 0-9, + blank 
num_channels = 1 # grayscale

batch_size = 64
patch_size = 5
depth1 = 32
depth2 = 64
depth3 = 128
num_hidden1 = 512
num_hidden2 = 128
num_hidden3 = 32
shape = [batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 32 x 32 x 11, filter size: 5 x 5 x 1 x 32
# P2: pooling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, filter size: 5 x 5 x 16 x 64
# P4: pooling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, filter size: 5 x 5 x 32 x 128
# Dropout
# F6: fully-connected layer, weight size: 64 x 11
# Output layer, weight size: 16 x 10

train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

    # Input data placeholders.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    
    layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    
    layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')

    s1_w1 = tf.get_variable("WS11", shape=[num_hidden1, num_hidden2],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s1_b1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='BS11')
    
    s2_w1 = tf.get_variable("WS12", shape=[num_hidden1, num_hidden2],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s2_b1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='BS12')
    
    s3_w1 = tf.get_variable("WS13", shape=[num_hidden1, num_hidden2],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s3_b1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='BS13')
    
    s4_w1 = tf.get_variable("WS14", shape=[num_hidden1, num_hidden2],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s4_b1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='BS14')
    
    s5_w1 = tf.get_variable("WS15", shape=[num_hidden1, num_hidden2],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s5_b1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='BS15')
    

    s1_w2 = tf.get_variable("WS21", shape=[num_hidden2, num_hidden3],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s1_b2 = tf.Variable(tf.constant(1.0, shape=[num_hidden3]), name='BS21')
    
    s2_w2 = tf.get_variable("WS22", shape=[num_hidden2, num_hidden3],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s2_b2 = tf.Variable(tf.constant(1.0, shape=[num_hidden3]), name='BS22')
    
    s3_w2 = tf.get_variable("WS23", shape=[num_hidden2, num_hidden3],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s3_b2 = tf.Variable(tf.constant(1.0, shape=[num_hidden3]), name='BS23')
    
    s4_w2 = tf.get_variable("WS24", shape=[num_hidden2, num_hidden3],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s4_b2 = tf.Variable(tf.constant(1.0, shape=[num_hidden3]), name='BS24')
    
    s5_w2 = tf.get_variable("WS25", shape=[num_hidden2, num_hidden3],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s5_b2 = tf.Variable(tf.constant(1.0, shape=[num_hidden3]), name='BS25')
    

    s1_w3 = tf.get_variable("WS31", shape=[num_hidden3, num_labels],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s1_b3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS31')
    
    s2_w3 = tf.get_variable("WS32", shape=[num_hidden3, num_labels],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s2_b3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS32')
    
    s3_w3 = tf.get_variable("WS33", shape=[num_hidden3, num_labels],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s3_b3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS33')
    
    s4_w3 = tf.get_variable("WS34", shape=[num_hidden3, num_labels],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s4_b3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS34')
    
    s5_w3 = tf.get_variable("WS35", shape=[num_hidden3, num_labels],\
                           initializer=tf.contrib.layers.xavier_initializer())
    s5_b3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS35')

    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P2')
        
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P4')
        
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')        
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        #print(reshape.get_shape())
        # hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        hidden11 = tf.nn.relu(tf.matmul(reshape, s1_w1) + s1_b1)
        hidden21 = tf.nn.relu(tf.matmul(reshape, s2_w1) + s2_b1)
        hidden31 = tf.nn.relu(tf.matmul(reshape, s3_w1) + s3_b1)
        hidden41 = tf.nn.relu(tf.matmul(reshape, s4_w1) + s4_b1)
        hidden51 = tf.nn.relu(tf.matmul(reshape, s5_w1) + s5_b1)
        
        hidden12 = tf.nn.relu(tf.matmul(hidden11, s1_w2) + s1_b2)
        hidden22 = tf.nn.relu(tf.matmul(hidden21, s2_w2) + s2_b2)
        hidden32 = tf.nn.relu(tf.matmul(hidden31, s3_w2) + s3_b2)
        hidden42 = tf.nn.relu(tf.matmul(hidden41, s4_w2) + s4_b2)
        hidden52 = tf.nn.relu(tf.matmul(hidden51, s5_w2) + s5_b2)
        
        logits1 = tf.matmul(hidden12, s1_w3) + s1_b3
        logits2 = tf.matmul(hidden22, s2_w3) + s2_b3
        logits3 = tf.matmul(hidden32, s3_w3) + s3_b3
        logits4 = tf.matmul(hidden42, s4_w3) + s4_b3
        logits5 = tf.matmul(hidden52, s5_w3) + s5_b3
        
        return [logits1, logits2, logits3, logits4, logits5]

    # Training computation.
    [logits1, logits2, logits3, logits4, logits5] = model(tf_train_dataset, 0.6, shape)
    
    # L2 regularization for the fully connected parameters.
    regularizers = [(tf.nn.l2_loss(s1_w2) + tf.nn.l2_loss(s1_b2)),
                    (tf.nn.l2_loss(s2_w2) + tf.nn.l2_loss(s2_b2)),
                    (tf.nn.l2_loss(s3_w2) + tf.nn.l2_loss(s3_b2)),
                    (tf.nn.l2_loss(s4_w2) + tf.nn.l2_loss(s4_b2)),
                    (tf.nn.l2_loss(s5_w2) + tf.nn.l2_loss(s5_b2)),]
    
    post_logits = [tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, tf_train_labels[:,5]))]
    
    loss = (post_logits[0] + 4e-4 * regularizers[0]) + (post_logits[1] + 4e-4 * regularizers[1]) + \
           (post_logits[2] + 4e-4 * regularizers[2]) + (post_logits[3] + 4e-4 * regularizers[3]) + \
           (post_logits[4] + 4e-4 * regularizers[4])
    
    # Optimizer.
    batch = tf.Variable(0, dtype=tf.float32)
    
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(0.01, batch * batch_size, train_size, 0.95, staircase=True)
    
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    
    # Predictions for the training, validation and test data.
    train_logits = model(tf_train_dataset, 1.0, shape)
    train_prediction = tf.pack([tf.nn.softmax(train_logits[0]), tf.nn.softmax(train_logits[1]),\
                                tf.nn.softmax(train_logits[2]), tf.nn.softmax(train_logits[3]),\
                                tf.nn.softmax(train_logits[4])])
    
    valid_logits = model(tf_valid_dataset, 1.0, shape)
    valid_prediction = tf.pack([tf.nn.softmax(valid_logits[0]), tf.nn.softmax(valid_logits[1]),\
                                tf.nn.softmax(valid_logits[2]), tf.nn.softmax(valid_logits[3]),\
                                tf.nn.softmax(valid_logits[4])])
    
    test_logits = model(tf_test_dataset, 1.0, shape)
    test_prediction = tf.pack([tf.nn.softmax(test_logits[0]), tf.nn.softmax(test_logits[1]),\
                               tf.nn.softmax(test_logits[2]), tf.nn.softmax(test_logits[3]),\
                               tf.nn.softmax(test_logits[4])])

    saver = tf.train.Saver()

In [12]:
num_steps = 25001

loss_values = []
train_accuracies = []
validation_accuracies = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        #if step % 10 == 0:
        #    loss_values.append(l)
            
        if (step % 1000 == 0): 
            train_accuracy = accuracy(predictions, batch_labels[:,1:6])
            valid_accuracy = accuracy(valid_prediction.eval(), valid_labels[:,1:6])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)    
            print('Validation accuracy: %.1f%%' % valid_accuracy)
            train_accuracies.append(train_accuracy)
            validation_accuracies.append(valid_accuracy)
            
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))
    
    save_path = saver.save(session, "cnn.convnet")
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 21.470789
Minibatch accuracy: 21.6%
Validation accuracy: 52.1%
Minibatch loss at step 1000: 6.032491
Minibatch accuracy: 61.6%
Validation accuracy: 58.5%
Minibatch loss at step 2000: 2.941924
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 1.670177
Minibatch accuracy: 92.2%
Validation accuracy: 89.1%
Minibatch loss at step 4000: 1.864727
Minibatch accuracy: 90.9%
Validation accuracy: 91.5%
Minibatch loss at step 5000: 0.914149
Minibatch accuracy: 97.5%
Validation accuracy: 92.8%
Minibatch loss at step 6000: 1.433883
Minibatch accuracy: 94.7%
Validation accuracy: 93.5%
Minibatch loss at step 7000: 1.208452
Minibatch accuracy: 95.9%
Validation accuracy: 94.4%
Minibatch loss at step 8000: 1.144454
Minibatch accuracy: 95.9%
Validation accuracy: 94.7%
Minibatch loss at step 9000: 0.647428
Minibatch accuracy: 97.5%
Validation accuracy: 94.8%
Minibatch loss at step 10000: 0.704464
Minibatch accuracy: 95.3%
Validation accu

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(1)

plt.bar(np.arange(0,10001,10), loss_values, align='center')
plt.title('Dropout = 0.9375')
plt.xlabel('Training Step')
plt.yticks(np.arange(0,21,5))
plt.xticks(np.arange(0,10500,5000))
plt.ylabel('Loss Value')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(2)

plt.plot(np.arange(0,10001,1000), train_accuracies, label = 'Training Accuracy')
plt.plot(np.arange(0,10001,1000), validation_accuracies, label = 'Validation Accuracy')
plt.legend(loc='lower right')

plt.title('Accuracy improvement on training phase')
plt.xlabel('Training Step')
plt.ylabel('Accuracy, %')

plt.show()

In [ ]:
from PIL import Image
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (20.0, 20.0)
f, ax = plt.subplots(nrows=1, ncols=10)

im_samples = []
for i, item in enumerate(np.sort(np.random.randint(0, test_labels.shape[0], size=10))):
    filename = test_filenames[item]
    fullname = os.path.join('test', filename)
    im = Image.open(fullname)
    house_num = ''
    for k in np.arange(test_labels[item,0]):
        house_num += str(test_labels[item,k+1])
        
    im_samples.extend([item])
    ax[i].axis('off')
    ax[i].set_title(house_num, loc='center')
    ax[i].imshow(im)

In [ ]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_test_dataset = tf.placeholder(tf.float32, shape=(10, 32, 32, 1))

    # Variables.
    layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')

    s1_w = tf.get_variable("WS1", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s1_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS1')
    s2_w = tf.get_variable("WS2", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s2_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS2')
    s3_w = tf.get_variable("WS3", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s3_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS3')
    s4_w = tf.get_variable("WS4", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s4_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS4')
    s5_w = tf.get_variable("WS5", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s5_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS5')
  
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S2')
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S4')
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        logits1 = tf.matmul(reshape, s1_w) + s1_b
        logits2 = tf.matmul(reshape, s2_w) + s2_b
        logits3 = tf.matmul(reshape, s3_w) + s3_b
        logits4 = tf.matmul(reshape, s4_w) + s4_b
        logits5 = tf.matmul(reshape, s5_w) + s5_b
        return [logits1, logits2, logits3, logits4, logits5]
  
    # Training computation.
    [logits1, logits2, logits3, logits4, logits5] = model(tf_test_dataset, 1, [10, 32, 32, 1])

    predict = tf.pack([tf.nn.softmax(logits1),tf.nn.softmax(logits2),tf.nn.softmax(logits3),\
                       tf.nn.softmax(logits4),tf.nn.softmax(logits5)])
    test_prediction = tf.transpose(tf.argmax(predict, 2))
    saver = tf.train.Saver()

with tf.Session(graph=graph) as session:
    saver.restore(session, "cnn.convnet")
    print("Model restored.")  

    print('Initialized')
    test_prediction = session.run(test_prediction, feed_dict={tf_test_dataset : test_dataset[im_samples,:,:,:],})
    print(test_prediction)

In [ ]:
plt.rcParams['figure.figsize'] = (15.0, 15.0)
f, ax = plt.subplots(nrows=1, ncols=10)

for i, j in enumerate(im_samples):
    filename = test_filenames[j]
    fullname = os.path.join('test', filename)
    im = Image.open(fullname)
    house_num = ''
    for k in np.arange(np.sum(test_prediction[i,:] != 10)):
        house_num += str(test_prediction[i,k])

    ax[i].axis('off')
    ax[i].set_title(house_num, loc='center')
    ax[i].imshow(im)